# Segment-Level Validation: Growth Table CSV vs Fabric Warehouse

This notebook performs **segment-level validation** by comparing aggregated metrics between CSV and Fabric data.

**Validation Segments:**
1. By Date
2. By Campaign
3. By Gender
4. By Age Group
5. By Campaign + Date
6. By Campaign + Gender
7. By Date + Gender + Age

**Files:**
- CSV: `merged_age_gender.csv` (Growth table)
- Excel: `merged_age_gender.xlsx` (Fabric export)

## Step 1: Install Required Package & Import Libraries

In [24]:
# Install openpyxl if needed
import sys
!{sys.executable} -m pip install openpyxl -q

import pandas as pd
import numpy as np
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Libraries imported successfully")
print(f"Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✓ Libraries imported successfully
Analysis started: 2025-12-17 10:13:44



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\Krishnadev\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Step 2: Load and Prepare Data

In [25]:
# Load CSV (skip 2 header rows)
print("Loading CSV...")
csv_df = pd.read_csv("merged_age_gender(growth).csv", skiprows=2)

# Clean and map columns
csv_df['Impr.'] = csv_df['Impr.'].str.replace(',', '').astype(int)
csv_df['Cost'] = pd.to_numeric(csv_df['Cost'], errors='coerce')
csv_df['Clicks'] = pd.to_numeric(csv_df['Clicks'], errors='coerce')

csv_df = csv_df.rename(columns={
    'Campaign': 'campaign_name',
    'Day': 'day',
    'Gender': 'gender',
    'Age': 'age',
    'Cost': 'cost',
    'Impr.': 'impressions',
    'Clicks': 'clicks'
})

print(f"✓ CSV loaded: {len(csv_df):,} rows")

# Load Fabric Excel
print("\nLoading Fabric export...")
fabric_df = pd.read_excel("merged_age_gender(gold)2.xlsx")
fabric_df['day'] = pd.to_datetime(fabric_df['day']).dt.strftime('%Y-%m-%d')

print(f"✓ Fabric loaded: {len(fabric_df):,} rows")

print("\n" + "="*80)
print("DATA SUMMARY")
print("="*80)
print(f"\nCSV Date Range: {csv_df['day'].min()} to {csv_df['day'].max()}")
print(f"Fabric Date Range: {fabric_df['day'].min()} to {fabric_df['day'].max()}")

Loading CSV...
✓ CSV loaded: 1,302 rows

Loading Fabric export...
✓ Fabric loaded: 6,472 rows

DATA SUMMARY

CSV Date Range: 2025-11-01 to 2025-11-30
Fabric Date Range: 2025-11-01 to 2025-11-30


## Step 3: Overall Totals Comparison

In [26]:
print("="*80)
print("OVERALL TOTALS COMPARISON")
print("="*80)

# Calculate totals
csv_totals = csv_df[['cost', 'impressions', 'clicks']].sum()
fabric_totals = fabric_df[['cost', 'impressions', 'clicks']].sum()

# Create comparison dataframe
overall_comparison = pd.DataFrame({
    'Metric': ['Cost (₹)', 'Impressions', 'Clicks'],
    'CSV': [csv_totals['cost'], csv_totals['impressions'], csv_totals['clicks']],
    'Fabric': [fabric_totals['cost'], fabric_totals['impressions'], fabric_totals['clicks']],
})

overall_comparison['Difference'] = overall_comparison['CSV'] - overall_comparison['Fabric']
overall_comparison['Diff %'] = (overall_comparison['Difference'] / overall_comparison['Fabric'] * 100).round(2)
overall_comparison['Match'] = overall_comparison['Difference'].abs() < 0.01

display(overall_comparison)

# Summary
matches = overall_comparison['Match'].sum()
print(f"\n✓ Perfect matches: {matches}/3 metrics")
if matches == 3:
    print("✓✓✓ ALL OVERALL TOTALS MATCH! ✓✓✓")
else:
    print(f"⚠ {3-matches} metric(s) have differences")

OVERALL TOTALS COMPARISON


,Metric,CSV,Fabric,Difference,Diff %,Match
0,Cost (₹),260312.70,260320.48,-7.78,-0.00,False
1,Impressions,1338526.00,1335418.00,3108.00,0.23,False
2,Clicks,52059.00,50129.00,1930.00,3.85,False



✓ Perfect matches: 0/3 metrics
⚠ 3 metric(s) have differences


## Step 4: Validation by Date

In [27]:
print("="*80)
print("SEGMENT VALIDATION: BY DATE")
print("="*80)

# Aggregate by date
csv_by_date = csv_df.groupby('day').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
csv_by_date.columns = ['day', 'cost_csv', 'impressions_csv', 'clicks_csv']

fabric_by_date = fabric_df.groupby('day').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
fabric_by_date.columns = ['day', 'cost_fabric', 'impressions_fabric', 'clicks_fabric']

# Merge and compare
date_comparison = pd.merge(csv_by_date, fabric_by_date, on='day', how='outer', indicator=True)

# Calculate differences
date_comparison['cost_diff'] = date_comparison['cost_csv'] - date_comparison['cost_fabric']
date_comparison['impr_diff'] = date_comparison['impressions_csv'] - date_comparison['impressions_fabric']
date_comparison['clicks_diff'] = date_comparison['clicks_csv'] - date_comparison['clicks_fabric']

# Check matches
date_comparison['cost_match'] = date_comparison['cost_diff'].abs() < 0.01
date_comparison['impr_match'] = date_comparison['impr_diff'].abs() < 0.01
date_comparison['clicks_match'] = date_comparison['clicks_diff'].abs() < 0.01
date_comparison['perfect_match'] = date_comparison['cost_match'] & date_comparison['impr_match'] & date_comparison['clicks_match']

# Display results
display_cols = ['day', 'cost_csv', 'cost_fabric', 'cost_diff', 
                'impressions_csv', 'impressions_fabric', 'impr_diff',
                'clicks_csv', 'clicks_fabric', 'clicks_diff', 'perfect_match']

print(f"\nTotal dates compared: {len(date_comparison)}")
print(f"Perfect matches: {date_comparison['perfect_match'].sum()}")
print(f"Dates with differences: {(~date_comparison['perfect_match']).sum()}")

print("\nDetailed comparison:")
display(date_comparison[display_cols].sort_values('day'))

# Save mismatches
if (~date_comparison['perfect_match']).sum() > 0:
    mismatches = date_comparison[~date_comparison['perfect_match']]
    mismatches[display_cols].to_csv('segment_validation_by_date.csv', index=False)
    print("\n✓ Date-level mismatches saved to: segment_validation_by_date.csv")

SEGMENT VALIDATION: BY DATE

Total dates compared: 29
Perfect matches: 0
Dates with differences: 29

Detailed comparison:


,day,cost_csv,cost_fabric,cost_diff,impressions_csv,impressions_fabric,impr_diff,clicks_csv,clicks_fabric,clicks_diff,perfect_match
0,2025-11-01,5406.62,5406.67,-0.05,89790,89680,110,1519,1428,91,False
1,2025-11-03,958.94,958.92,0.02,6534,6422,112,190,137,53,False
2,2025-11-04,5404.33,5404.34,-0.01,57103,56981,122,1475,1392,83,False
3,2025-11-05,576.05,576.02,0.03,7172,7064,108,144,102,42,False
4,2025-11-06,2212.80,2212.79,0.01,22750,22634,116,564,501,63,False
5,2025-11-07,5691.94,5691.93,0.01,49953,49838,115,1396,1317,79,False
6,2025-11-08,10985.24,10985.25,-0.01,84543,84437,106,1955,1872,83,False
7,2025-11-09,16887.90,16887.89,0.01,113292,113181,111,2092,2017,75,False
8,2025-11-10,16130.80,16130.81,-0.01,90208,90093,115,1838,1742,96,False
9,2025-11-11,11965.48,11965.46,0.02,87841,87738,103,1773,1700,73,False



✓ Date-level mismatches saved to: segment_validation_by_date.csv


## Step 5: Validation by Campaign

In [28]:
print("="*80)
print("SEGMENT VALIDATION: BY CAMPAIGN")
print("="*80)

# Aggregate by campaign
csv_by_campaign = csv_df.groupby('campaign_name').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
csv_by_campaign.columns = ['campaign_name', 'cost_csv', 'impressions_csv', 'clicks_csv']

fabric_by_campaign = fabric_df.groupby('campaign_name').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
fabric_by_campaign.columns = ['campaign_name', 'cost_fabric', 'impressions_fabric', 'clicks_fabric']

# Merge and compare
campaign_comparison = pd.merge(csv_by_campaign, fabric_by_campaign, on='campaign_name', how='outer', indicator=True)

# Calculate differences
campaign_comparison['cost_diff'] = campaign_comparison['cost_csv'] - campaign_comparison['cost_fabric']
campaign_comparison['impr_diff'] = campaign_comparison['impressions_csv'] - campaign_comparison['impressions_fabric']
campaign_comparison['clicks_diff'] = campaign_comparison['clicks_csv'] - campaign_comparison['clicks_fabric']

# Check matches
campaign_comparison['perfect_match'] = (
    (campaign_comparison['cost_diff'].abs() < 0.01) & 
    (campaign_comparison['impr_diff'].abs() < 0.01) & 
    (campaign_comparison['clicks_diff'].abs() < 0.01)
)

# Display results
display_cols = ['campaign_name', 'cost_csv', 'cost_fabric', 'cost_diff',
                'impressions_csv', 'impressions_fabric', 'impr_diff',
                'clicks_csv', 'clicks_fabric', 'clicks_diff', 'perfect_match']

print(f"\nTotal campaigns compared: {len(campaign_comparison)}")
print(f"Perfect matches: {campaign_comparison['perfect_match'].sum()}")
print(f"Campaigns with differences: {(~campaign_comparison['perfect_match']).sum()}")

print("\nDetailed comparison:")
display(campaign_comparison[display_cols].sort_values('campaign_name'))

# Save mismatches
if (~campaign_comparison['perfect_match']).sum() > 0:
    mismatches = campaign_comparison[~campaign_comparison['perfect_match']]
    mismatches[display_cols].to_csv('segment_validation_by_campaign.csv', index=False)
    print("\n✓ Campaign-level mismatches saved to: segment_validation_by_campaign.csv")

SEGMENT VALIDATION: BY CAMPAIGN

Total campaigns compared: 5
Perfect matches: 0
Campaigns with differences: 5

Detailed comparison:


,campaign_name,cost_csv,cost_fabric,cost_diff,impressions_csv,impressions_fabric,impr_diff,clicks_csv,clicks_fabric,clicks_diff,perfect_match
0,Cadiveu_Instamart_External_20th_Nov_2025,5499.50,5499.49,0.01,342,268,74,26,18,8,False
1,IKONIC-AMZ-Glide-Peach-14-Oct-2025,30429.60,30439.37,-9.77,287833,287868,-35,10622,10625,-3,False
2,ME_Search_|_Oct_25,111296.45,111296.54,-0.09,646629,645180,1449,13091,12082,1009,False
3,Nykaa_Black_Friday_Traffic,3499.34,3497.13,2.21,216816,216752,64,16089,16010,79,False
4,PRO_Search_|_Oct_25,109587.81,109587.95,-0.14,186906,185350,1556,12231,11394,837,False



✓ Campaign-level mismatches saved to: segment_validation_by_campaign.csv


## Step 6: Validation by Gender

In [29]:
print("="*80)
print("SEGMENT VALIDATION: BY GENDER")
print("="*80)

# Aggregate by gender
csv_by_gender = csv_df.groupby('gender').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
csv_by_gender.columns = ['gender', 'cost_csv', 'impressions_csv', 'clicks_csv']

fabric_by_gender = fabric_df.groupby('gender').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
fabric_by_gender.columns = ['gender', 'cost_fabric', 'impressions_fabric', 'clicks_fabric']

# Merge and compare
gender_comparison = pd.merge(csv_by_gender, fabric_by_gender, on='gender', how='outer')

# Calculate differences
gender_comparison['cost_diff'] = gender_comparison['cost_csv'] - gender_comparison['cost_fabric']
gender_comparison['impr_diff'] = gender_comparison['impressions_csv'] - gender_comparison['impressions_fabric']
gender_comparison['clicks_diff'] = gender_comparison['clicks_csv'] - gender_comparison['clicks_fabric']

# Check matches
gender_comparison['perfect_match'] = (
    (gender_comparison['cost_diff'].abs() < 0.01) & 
    (gender_comparison['impr_diff'].abs() < 0.01) & 
    (gender_comparison['clicks_diff'].abs() < 0.01)
)

print(f"\nPerfect matches: {gender_comparison['perfect_match'].sum()}/{len(gender_comparison)}")
display(gender_comparison)

SEGMENT VALIDATION: BY GENDER

Perfect matches: 0/6


,gender,cost_csv,impressions_csv,clicks_csv,cost_fabric,impressions_fabric,clicks_fabric,cost_diff,impr_diff,clicks_diff,perfect_match
0,FEMALE,NaN,NaN,NaN,128266.23,664996.00,25289.00,NaN,NaN,NaN,False
1,Female,128257.29,666019.00,25945.00,NaN,NaN,NaN,NaN,NaN,NaN,False
2,MALE,NaN,NaN,NaN,68162.33,420655.00,16561.00,NaN,NaN,NaN,False
3,Male,68163.55,421691.00,17199.00,NaN,NaN,NaN,NaN,NaN,NaN,False
4,UNDETERMINED,NaN,NaN,NaN,63891.92,249767.00,8279.00,NaN,NaN,NaN,False
5,Unknown,63891.86,250816.00,8915.00,NaN,NaN,NaN,NaN,NaN,NaN,False


## Step 7: Validation by Age Group

In [30]:
print("="*80)
print("SEGMENT VALIDATION: BY AGE GROUP")
print("="*80)

# Aggregate by age
csv_by_age = csv_df.groupby('age').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
csv_by_age.columns = ['age', 'cost_csv', 'impressions_csv', 'clicks_csv']

fabric_by_age = fabric_df.groupby('age').agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
fabric_by_age.columns = ['age', 'cost_fabric', 'impressions_fabric', 'clicks_fabric']

# Merge and compare
age_comparison = pd.merge(csv_by_age, fabric_by_age, on='age', how='outer')

# Calculate differences
age_comparison['cost_diff'] = age_comparison['cost_csv'] - age_comparison['cost_fabric']
age_comparison['impr_diff'] = age_comparison['impressions_csv'] - age_comparison['impressions_fabric']
age_comparison['clicks_diff'] = age_comparison['clicks_csv'] - age_comparison['clicks_fabric']

# Check matches
age_comparison['perfect_match'] = (
    (age_comparison['cost_diff'].abs() < 0.01) & 
    (age_comparison['impr_diff'].abs() < 0.01) & 
    (age_comparison['clicks_diff'].abs() < 0.01)
)

print(f"\nPerfect matches: {age_comparison['perfect_match'].sum()}/{len(age_comparison)}")
display(age_comparison)

SEGMENT VALIDATION: BY AGE GROUP

Perfect matches: 0/13


,age,cost_csv,impressions_csv,clicks_csv,cost_fabric,impressions_fabric,clicks_fabric,cost_diff,impr_diff,clicks_diff,perfect_match
0,18 - 24,45615.77,270832.00,9324.00,NaN,NaN,NaN,NaN,NaN,NaN,False
1,18-24,NaN,NaN,NaN,45622.50,270369.00,8997.00,NaN,NaN,NaN,False
2,25 - 34,95556.48,479613.00,18357.00,NaN,NaN,NaN,NaN,NaN,NaN,False
3,25-34,NaN,NaN,NaN,95559.48,479140.00,17993.00,NaN,NaN,NaN,False
4,35 - 44,39165.50,228410.00,8823.00,NaN,NaN,NaN,NaN,NaN,NaN,False
5,35-44,NaN,NaN,NaN,39164.34,227943.00,8480.00,NaN,NaN,NaN,False
6,45 - 54,11459.89,59095.00,3467.00,NaN,NaN,NaN,NaN,NaN,NaN,False
7,45-54,NaN,NaN,NaN,11459.57,58620.00,3216.00,NaN,NaN,NaN,False
8,55 - 64,2703.13,28642.00,1812.00,NaN,NaN,NaN,NaN,NaN,NaN,False
9,55-64,NaN,NaN,NaN,2702.92,28258.00,1657.00,NaN,NaN,NaN,False


## Step 8: Validation by Campaign + Date

In [31]:
print("="*80)
print("SEGMENT VALIDATION: BY CAMPAIGN + DATE")
print("="*80)

# Aggregate by campaign and date
csv_by_camp_date = csv_df.groupby(['campaign_name', 'day']).agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
csv_by_camp_date.columns = ['campaign_name', 'day', 'cost_csv', 'impressions_csv', 'clicks_csv']

fabric_by_camp_date = fabric_df.groupby(['campaign_name', 'day']).agg({
    'cost': 'sum',
    'impressions': 'sum',
    'clicks': 'sum'
}).reset_index()
fabric_by_camp_date.columns = ['campaign_name', 'day', 'cost_fabric', 'impressions_fabric', 'clicks_fabric']

# Merge and compare
camp_date_comparison = pd.merge(csv_by_camp_date, fabric_by_camp_date, 
                                 on=['campaign_name', 'day'], how='outer', indicator=True)

# Calculate differences
camp_date_comparison['cost_diff'] = camp_date_comparison['cost_csv'] - camp_date_comparison['cost_fabric']
camp_date_comparison['impr_diff'] = camp_date_comparison['impressions_csv'] - camp_date_comparison['impressions_fabric']
camp_date_comparison['clicks_diff'] = camp_date_comparison['clicks_csv'] - camp_date_comparison['clicks_fabric']

# Check matches
camp_date_comparison['perfect_match'] = (
    (camp_date_comparison['cost_diff'].abs() < 0.01) & 
    (camp_date_comparison['impr_diff'].abs() < 0.01) & 
    (camp_date_comparison['clicks_diff'].abs() < 0.01)
)

print(f"\nTotal campaign+date segments: {len(camp_date_comparison)}")
print(f"Perfect matches: {camp_date_comparison['perfect_match'].sum()}")
print(f"Segments with differences: {(~camp_date_comparison['perfect_match']).sum()}")

# Show mismatches only
if (~camp_date_comparison['perfect_match']).sum() > 0:
    print("\nSegments with differences:")
    mismatches = camp_date_comparison[~camp_date_comparison['perfect_match']]
    display(mismatches[['campaign_name', 'day', 'cost_diff', 'impr_diff', 'clicks_diff']].head(20))
    
    mismatches.to_csv('segment_validation_by_campaign_date.csv', index=False)
    print("\n✓ Campaign+Date mismatches saved to: segment_validation_by_campaign_date.csv")
else:
    print("\n✓✓✓ ALL CAMPAIGN+DATE SEGMENTS MATCH PERFECTLY! ✓✓✓")

SEGMENT VALIDATION: BY CAMPAIGN + DATE

Total campaign+date segments: 104
Perfect matches: 24
Segments with differences: 80

Segments with differences:


,campaign_name,day,cost_diff,impr_diff,clicks_diff
1,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-21,0.00,10,0
2,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-22,0.00,6,0
3,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-23,0.00,9,0
4,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-24,0.00,7,2
5,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-25,0.00,8,2
6,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-26,0.00,7,0
7,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-27,0.01,11,2
8,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-28,0.00,4,0
9,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-29,0.00,6,0
10,Cadiveu_Instamart_External_20th_Nov_2025,2025-11-30,0.00,6,2



✓ Campaign+Date mismatches saved to: segment_validation_by_campaign_date.csv

## Step 9: Final Summary Report

In [32]:
print("="*80)
print("SEGMENT VALIDATION SUMMARY REPORT")
print("="*80)
print(f"\nAnalysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Create summary table
summary_data = [
    ['Overall Totals', 1, overall_comparison['Match'].sum(), 3 - overall_comparison['Match'].sum()],
    ['By Date', len(date_comparison), date_comparison['perfect_match'].sum(), 
     (~date_comparison['perfect_match']).sum()],
    ['By Campaign', len(campaign_comparison), campaign_comparison['perfect_match'].sum(), 
     (~campaign_comparison['perfect_match']).sum()],
    ['By Gender', len(gender_comparison), gender_comparison['perfect_match'].sum(), 
     (~gender_comparison['perfect_match']).sum()],
    ['By Age', len(age_comparison), age_comparison['perfect_match'].sum(), 
     (~age_comparison['perfect_match']).sum()],
    ['By Campaign+Date', len(camp_date_comparison), camp_date_comparison['perfect_match'].sum(), 
     (~camp_date_comparison['perfect_match']).sum()]
]

summary_df = pd.DataFrame(summary_data, 
                         columns=['Segment Type', 'Total Segments', 'Matches', 'Mismatches'])
summary_df['Match %'] = (summary_df['Matches'] / summary_df['Total Segments'] * 100).round(2)

print("\n")
display(summary_df)

# Overall assessment
total_segments = summary_df['Total Segments'].sum()
total_matches = summary_df['Matches'].sum()
overall_match_pct = (total_matches / total_segments * 100)

print("\n" + "="*80)
print(f"OVERALL MATCH RATE: {total_matches}/{total_segments} ({overall_match_pct:.1f}%)")
print("="*80)

if overall_match_pct == 100:
    print("\n✓✓✓ PERFECT VALIDATION! All segments match! ✓✓✓")
elif overall_match_pct >= 95:
    print("\n✓ EXCELLENT! Minor discrepancies found.")
elif overall_match_pct >= 80:
    print("\n⚠ GOOD: Some discrepancies need review.")
else:
    print("\n⚠ ATTENTION: Significant discrepancies found. Review required.")

print("\n" + "-"*80)
print("GENERATED FILES:")
print("-"*80)
files = []
if (~date_comparison['perfect_match']).sum() > 0:
    files.append("✓ segment_validation_by_date.csv")
if (~campaign_comparison['perfect_match']).sum() > 0:
    files.append("✓ segment_validation_by_campaign.csv")
if (~camp_date_comparison['perfect_match']).sum() > 0:
    files.append("✓ segment_validation_by_campaign_date.csv")

if files:
    for f in files:
        print(f)
else:
    print("No mismatch files generated - all segments match!")

print("\n" + "="*80)
print("VALIDATION COMPLETE")
print("="*80)

SEGMENT VALIDATION SUMMARY REPORT

Analysis completed: 2025-12-17 10:13:45




,Segment Type,Total Segments,Matches,Mismatches,Match %
0,Overall Totals,1,0,3,0.00
1,By Date,29,0,29,0.00
2,By Campaign,5,0,5,0.00
3,By Gender,6,0,6,0.00
4,By Age,13,0,13,0.00
5,By Campaign+Date,104,24,80,23.08



OVERALL MATCH RATE: 24/158 (15.2%)

⚠ ATTENTION: Significant discrepancies found. Review required.

--------------------------------------------------------------------------------
GENERATED FILES:
--------------------------------------------------------------------------------
✓ segment_validation_by_date.csv
✓ segment_validation_by_campaign.csv
✓ segment_validation_by_campaign_date.csv

VALIDATION COMPLETE
